In [13]:
# Necessary imports
from langchain_community.tools import DuckDuckGoSearchRun, StructuredTool
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field
import requests
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

# Load api key
load_dotenv()

True

In [14]:
# Model config
model = ChatGroq(model="llama-3.1-8b-instant")

In [15]:
# Tool
search_tool = DuckDuckGoSearchRun()

# Input argument schema
class WeatherTool(BaseModel):
    city: str = Field(required=True, description="Name of the city from where we fetch it's weather conditions.")

# Weather tool
def get_weather_conditions(city: str) -> str:
    """A weather tool which fetches weather conditons of the given city."""
    url = f"https://api.weatherapi.com/v1/current.json?key=626c2425693948fea60110858260601&q={city}"
    response = requests.get(url=url)

    return response.json()

# Tool creation
weather_tool = StructuredTool.from_function(
    func=get_weather_conditions,
    args_schema=WeatherTool
)

# Tool binding
model = model.bind_tools(tools=[search_tool, weather_tool])

/var/folders/2n/vnp09xnn0s35m3xr_5c7_b300000gn/T/ipykernel_69921/2139124312.py:6: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  city: str = Field(required=True, description="Name of the city from where we fetch it's weather conditions.")


In [16]:
query = HumanMessage(content="What is the capital of India and what is it's current weather conditons.")

model_result = model.invoke(query.content)

for tool_call in model_result.tool_calls:
    if tool_call["name"] == "duckduckgo_search":
        tool_result = search_tool.invoke(tool_call["args"])
        print(tool_result)
    if tool_call["name"] == "get_weather_conditions":
        tool_result = weather_tool.invoke(tool_call["args"])
        print(tool_result)

{'location': {'name': 'New Delhi', 'region': 'Delhi', 'country': 'India', 'lat': 28.6, 'lon': 77.2, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1767699656, 'localtime': '2026-01-06 17:10'}, 'current': {'last_updated_epoch': 1767699000, 'last_updated': '2026-01-06 17:00', 'temp_c': 12.3, 'temp_f': 54.1, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 5.1, 'wind_kph': 8.3, 'wind_degree': 326, 'wind_dir': 'NNW', 'pressure_mb': 1021.0, 'pressure_in': 30.15, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 82, 'cloud': 50, 'feelslike_c': 11.7, 'feelslike_f': 53.0, 'windchill_c': 19.0, 'windchill_f': 66.2, 'heatindex_c': 19.0, 'heatindex_f': 66.2, 'dewpoint_c': 0.4, 'dewpoint_f': 32.8, 'vis_km': 1.1, 'vis_miles': 0.0, 'uv': 0.0, 'gust_mph': 8.8, 'gust_kph': 14.2, 'short_rad': 394.24, 'diff_rad': 97.71, 'dni': 478.13, 'gti': 179.94}}
New Delhi[a] is the capital of India and a part of the National Capital Territory 